### Build a Question Answering application over a Graph Database


In [8]:
NEO4J_URI="neo4j+s://ee03ca9e.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="Z8kYleTPoM3MCKfGbUXN6v3BLKxhmtgd4TarUwYbpuA"

In [9]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD


In [10]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

In [11]:
#dataset movie data
movie_query=""" 
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title  = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, "|") |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors,"|")| 
        MERGE (p:Person {name:trim(actor)})
        MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres,"|")| 
    MERGE (g:Genre {name:trim(genre)}) 
    MERGE (m)-[:IN_GENRE]->(g))

"""

In [12]:
movie_query

' \nLOAD CSV WITH HEADERS FROM \n\'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv\' as row\n\nMERGE(m:Movie{id:row.movieId})\nSET m.released = date(row.released),\n    m.title  = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, "|") |\n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor in split(row.actors,"|")| \n        MERGE (p:Person {name:trim(actor)})\n        MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre in split(row.genres,"|")| \n    MERGE (g:Genre {name:trim(genre)}) \n    MERGE (m)-[:IN_GENRE]->(g))\n\n'

In [14]:
graph.query(movie_query)

[]

In [15]:
graph.refresh_schema()
print(graph.schema)

Node properties:
User {city: STRING, name: STRING, userId: INTEGER, age: INTEGER}
Post {postId: INTEGER, content: STRING}
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:User)-[:POSTED]->(:Post)
(:User)-[:FRIENDS]->(:User)
(:User)-[:LIKES]->(:User)
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [16]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")



In [21]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.3-70b-versatile")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000282E49AE990>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000282E4D97920>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [23]:
from langchain_classic.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(graph=graph,llm=llm,allow_dangerous_requests=True,verbose=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x00000282E1F44980>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video

In [24]:
response = chain.invoke({"query":"Who was the director of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:DIRECTED]->(m:Movie {title: 'Casino'}) RETURN p.name


[#F7E7]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('si-ee03ca9e-c92d.production-orch-0064.neo4j.io', 7687)) (ResolvedIPv6Address(('64:ff9b::227e:406e', 7687, 0, 0))): OSError('No data')
Transaction failed and will be retried in 0.8400082747878893s (Failed to read from defunct connection IPv4Address(('si-ee03ca9e-c92d.production-orch-0064.neo4j.io', 7687)) (ResolvedIPv6Address(('64:ff9b::227e:406e', 7687, 0, 0))))


Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'Who was the director of the movie Casino',
 'result': 'Martin Scorsese was the director of the movie Casino.'}

In [25]:
response = chain.invoke({"query":"Who were the actors of movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(p:Person) RETURN p.name
Full Context:
[{'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}]

> Finished chain.


{'query': 'Who were the actors of movie Casino',
 'result': 'The actors of the movie Casino were Robert De Niro, Joe Pesci, Sharon Stone, and James Woods.'}